In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

#Getting Data

train = pd.read_csv('titanic_train.csv')
test = pd.read_csv('titanic_test.csv')

# Data Cleaning

# Defining a fuction to fulfill the missing data in age column
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age
    
# Applying to test and train data

train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)
test['Age'] = test[['Age','Pclass']].apply(impute_age,axis=1)
    
  
test['Fare'].loc[152]=test['Fare'].mean()
test['Fare'].loc[152]

# Cleaning Cabin column

train.drop('Cabin',axis=1,inplace=True)
test.drop('Cabin',axis=1,inplace=True)

## Converting Categorical Features and dropping unuseful args

train_new = pd.get_dummies(data=train,columns=['Sex'],drop_first=True)
#train_new = pd.get_dummies(data=train_new,columns=['Embarked'],drop_first=True)
train_new.drop(['Name','Ticket','Embarked'],axis=1,inplace=True)

## Converting Categorical Features and dropping unuseful args

test_new = pd.get_dummies(data=test,columns=['Sex'],drop_first=True)
test_new = pd.get_dummies(data=test_new,columns=['Embarked'],drop_first=True)
test_new.drop(['Name','Ticket'],axis=1,inplace=True)

C:\Users\Ruindows\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
import tensorflow as tf

X = train_new.drop(['Survived','PassengerId'],axis=1)
y = train_new['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))
    
classi = tf.estimator.DNNClassifier(hidden_units=[500,500,500,500,500,500,500,500,1000,1000,1000], n_classes=2,feature_columns=feat_cols)

input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,shuffle=True)

classi.train(input_fn=input_func,steps=100000)

pred_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

note_predictions = list(classi.predict(input_fn=pred_input_func))

final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])
    
print(confusion_matrix(y_test,final_preds))
print(classification_report(y_test,final_preds))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Ruindows\\AppData\\Local\\Temp\\tmpdr9gomz_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000061AD071C50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflo